In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dlib
import tensorflow as tf

In [2]:
detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [3]:
data = pd.read_csv('data/fer2013.csv')

In [34]:
# N = 1000
D = data

In [35]:
training_image = []
training_label_full = []
test_image = []
test_label_full = []


for index, row in D.iterrows():
    if row['Usage'] == 'Training':
        training_image.append(np.reshape(np.array(row['pixels'].split(), dtype=np.uint8), (48, 48)))
        training_label_full.append(row['emotion'])
    else:
        test_image.append(np.reshape(np.array(row['pixels'].split(), dtype=np.uint8), (48, 48)))
        test_label_full.append(row['emotion'])
        
# returns feature vector from an image
def get_landmarks(_image):
    image = _image.copy()
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
                
#         xmean = np.mean(xlist) #Get the mean of both axes to determine centre of gravity
#         ymean = np.mean(ylist)
        xmean = xlist[29]
        ymean = ylist[29]
        xcentral = [(x-xmean) for x in xlist] #get distance between each point and the central point in both axes
        ycentral = [(y-ymean) for y in ylist]
        
        # point 29 - nose tip
        # point 26 - middle point b/w two eyes
        cv.circle(image, (int(xlist[29]), int(ylist[29])), 1, (0,0,255))
        
        angle_nose = np.arctan2((ylist[26]-ymean), (xlist[26]-xmean)) * 180 / np.pi
        if angle_nose < 0:
            angle_nose += 90
        else:
            angle_nose -= 90
            
#         print(angle_nose)

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x) #Add the coordinates relative to the centre of gravity
            landmarks_vectorised.append(y)

            #Get the euclidean distance between each point and the centre point (the vector length)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)

            #Get the angle the vector describes relative to the image, corrected for the offset that the nosebrigde has when the face is not perfectly horizontal
            anglerelative = (np.arctan2((z - ymean), (w - xmean)) * 180 / np.pi) - angle_nose
            landmarks_vectorised.append(anglerelative)
        
    if len(detections) < 1: 
        landmarks_vectorised = "error"     
    
    return landmarks_vectorised

In [36]:
import time

def make_sets():
    training_data = []
    test_data = []
    training_label = []
    test_label = []
    
    #Append data to training and prediction list, and generate labels 0-7
    index = 0
    
    start_time = time.time()
    
    for item in training_image:
        landmarks_vectorised = get_landmarks(item)
        if landmarks_vectorised == "error":
            pass
        else:
            training_data.append(landmarks_vectorised) #append image array to training data list
            training_label.append(training_label_full[index])
            
        index += 1
        
    print('train - ', time.time() - start_time)
    
    index = 0
    for item in test_image:
        landmarks_vectorised = get_landmarks(item)
        if landmarks_vectorised == "error":
            pass
        else:
            test_data.append(landmarks_vectorised)
            test_label.append(test_label_full[index])
            
        index += 1
        
    print('test - ', time.time() - start_time)

    return training_data, test_data, training_label, test_label

# --------------------------------------------------------

train_data, test_data, training_label, test_label = make_sets()

print(len(train_data), len(training_label))

# pd.DataFrame(train_data)
# pd.DataFrame(test_data)

('train - ', 93.32631802558899)
('test - ', 116.11344814300537)
(19863, 19863)


In [37]:
train_data = np.array(train_data)
training_label = np.array(training_label)

test_data = np.array(test_data)
test_label = np.array(test_label)

train_label_one_hot = tf.Session().run(tf.one_hot(training_label, depth=7, on_value=1, off_value=0))
test_label_one_hot = tf.Session().run(tf.one_hot(test_label, depth=7, on_value=1, off_value=0))

In [38]:
# tensorflow
no_of_features = 268
no_of_classes = 7
x = tf.placeholder(tf.float32, [None, no_of_features])
W = tf.Variable(tf.zeros([no_of_features, no_of_classes]))
b = tf.Variable(tf.zeros([no_of_classes]))
y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, no_of_classes])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

start_time = time.time()
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

print('tf init time - ', time.time() - start_time)

('tf init time - ', 0.16815495491027832)


In [40]:
start_time = time.time()
for _ in range(1000):
#     batch_xs, batch_ys = mnist.train.next_batch(100)
    if _ % 50 == 0:
        print(_)
    sess.run(train_step, feed_dict={x: train_data, y_: train_label_one_hot})

print('tf run time - ', time.time() - start_time)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
('tf run time - ', 48.75536608695984)


In [41]:
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test_data, y_: test_label_one_hot}))

0.24549
